In [1]:
import pandas as pd
import os

# Define relative paths
INPUT_DIR = "../input"
OUTPUT_DIR = "../build"

# Input and output file paths
routers_list_path = os.path.join(INPUT_DIR, "sorted_routers_list.txt")
network_input_path = os.path.join(INPUT_DIR, "NetworkInput_BSNL.txt")
excel_path = os.path.join(INPUT_DIR, "Router_Coordinates_Jittered_Final.xlsx")
output_dat_path = os.path.join(OUTPUT_DIR, "edgenumbers.dat")
output_txt_path = os.path.join(OUTPUT_DIR, "edgenumbers.txt")

# Parse routers list
vertex_to_router = {}
with open(routers_list_path, 'r') as f:
    for line in f:
        vertex, router = line.strip().split(maxsplit=1)
        vertex_to_router[int(vertex)] = router

# Parse network input
edges = []
with open(network_input_path, 'r') as f:
    next(f)  # Skip header
    for line in f:
        parts = line.strip().split()
        edges.append((int(parts[0]), int(parts[1])))

# Load coordinates from Excel
df = pd.read_excel(excel_path)
coord_dict = {}
for _, row in df.iterrows():
    router, lat, lon = row['From_Router'], row['From_Latitude'], row['From_Longitude']
    coord_dict[router] = (lat, lon)
for _, row in df.iterrows():
    router, lat, lon = row['To_Router'], row['To_Latitude'], row['To_Longitude']
    if router not in coord_dict:
        coord_dict[router] = (lat, lon)

# Generate output content
output_content = (
    f"{'edgeno':<5} "
    f"{'vertex1':<5} "
    f"{'router_name1':<18} "
    f"{'from_Latitude':>12} "
    f"{'from_longtitude':>12} "
    f"{'vertex2':<5} "
    f"{'router_name2':<18} "
    f"{'to_latitude':>12} "
    f"{'to_longitude':>12}\n"
)

for i, (v1, v2) in enumerate(edges, 1):
    router1 = vertex_to_router.get(v1, "UNKNOWN")
    router2 = vertex_to_router.get(v2, "UNKNOWN")
    lat1, lon1 = coord_dict.get(router1, (0.0, 0.0))
    lat2, lon2 = coord_dict.get(router2, (0.0, 0.0))
    
    line = (
        f"{i:<5} "
        f"{v1:<5} "
        f"{router1:<18} "
        f"{lat1:>12.6f} "
        f"{lon1:>12.6f} "
        f"{v2:<5} "
        f"{router2:<18} "
        f"{lat2:>12.6f} "
        f"{lon2:>12.6f}\n"
    )
    output_content += line

# Write output to both .dat and .txt files
with open(output_dat_path, 'w') as f:
    f.write(output_content)

with open(output_txt_path, 'w') as f:
    f.write(output_content)